In [7]:
import glob
import pandas as pd
from shutil import copytree
import json
import pickle
import numpy as np

In [3]:
def random_dna_sequence(length):
    return ''.join(np.random.choice(('A', 'C', 'T', 'G')) for _ in range(length))

df = pd.read_csv('./all_genes.csv')

In [4]:
df[df['gene_id'] == 'BBF10K_003208']['seq'].tolist()

['AAACACGTGGCAAACATTCCGGTCTCGAATGAGCACGATCGAGGAACGCGTGAAGAAAATCATTGGCGAACAACTGGGTGTTAAACAAGAAGAGGTGACCAACAATGCCAGCTTTGTCGAGGACCTGGGCGCCGACAGCCTGGATACGGTTGAGCTGGTTATGGCATTGGAGGAAGAATTTGACACGGAGATTCCGGATGAAGAGGCTGAGAAAATTACTACCGTACAAGCGGCGATCGACTATATTAACGGCCATCAAGCCTGAAGAGCTTGGAGACCCGACTGCCGACATGATCTAT']

In [5]:
with open('./../examples/gene.json') as f:
    new_json = json.load(f)
    
tags = ['Bacillus subtilis', 'Mycoplasma pneumonia', 'Mycoplasma genitalium', 'Mesoplasma florum']
directories = {'Bacillus subtilis': './../collections/bacillus_essential/genes/', 
               'Mycoplasma pneumonia': './../collections/myco_pneumoniae/genes/',
               'Mycoplasma genitalium': './../collections/myco_genitalium/genes/', 
               'Mesoplasma florum': './../collections/meso_florum/genes/'}

In [6]:
# Copy genes from data directory into genes
for tag in tags:
    for directory in glob.glob('./../data/**'):
        json_file = glob.glob('{}/*.json'.format(directory))
        with open(json_file[0]) as f:
            data = json.load(f)
        if 'tags' in data:
            if tag in data['tags']:
                copytree(directory, '{}{}'.format(directories[tag],data['gene_id']))
                

In [7]:
#### RUN AFTER ADDGENE STUFF, PRIMERS NO GOOD

for json_file in glob.glob('./../collections/**/genes/**/*.json'):
    with open(json_file) as f:
        data = json.load(f)
    with open('./../examples/gene.json') as f:
        new_json = json.load(f)
    
    new_json['gene_id'] = data['gene_id']
    new_json['part_type'] = data['info']['gene_metadata']['cloning']['part_type']
    new_json['tags'] = data['tags'] + ['cds','moclo']
    new_json['original_sequence'] = data['sequence']['original_sequence']
    new_json['optimized_sequence'] = data['sequence']['optimized_sequence']
    new_json['status'] = 'Unapproved'
    new_json['author'] = [{'name': 'Keoni Gandall', 'email': 'koeng101@gmail.com', 'affiliation': 'BioBricks Foundation', 'orcid':'https://orcid.org/0000-0003-3995-6416'}]
    new_json['genbank'] = data['genbank']
    
    seq_list = df[df['gene_id'] == data['gene_id']]['seq'].tolist()
    if seq_list == []:
        new_json['status'] = 'Design_failed'
    else:
        seq = seq_list[0]
        if 'Bacillus subtilis' not in data['tags']:
            new_json['primer_for'] = seq[0:20]
            new_json['primer_rev'] = seq[-20:]
        else: 
            new_json['primer_for'] = ''
            new_json['primer_rev'] = ''
        if len(seq) < 300:
            seq = seq + random_dna_sequence(300-len(seq))
        new_json['synthesized_sequence'] = seq

    
    with open(json_file, 'w') as outfile:
        json.dump(new_json, outfile)

In [8]:
paths = ['meso_florum','myco_genitalium','myco_pneumoniae']

for path in paths:
    for json_file in glob.glob('./../collections/{}/genes/**/*.json'.format(path)):
        with open(json_file) as f:
            data = json.load(f)
        
        seq = data['synthesized_sequence']
        
        data['primer_for'] = seq[0:20] 
        end = seq.find('GAGACC')
        data['primer_rev'] = seq[end+6:end+26]
        
        with open(json_file, 'w') as outfile:
            json.dump(data, outfile)

'CCAGGAAAGTCGGCTAATCT'